In [16]:
import mdtraj as md
import numpy as np
import math
import os
import networkx as nx

from ele import element_from_symbol

# Hydrogen bond analysis by atomic species over a timeseries

This example will perform effectively the same analysis as the `hbonds_by_atom.ipynb` but instead of considering only a single frame, we will use the matrix output that will allow us to get averages of the hbonds, by type. 

While the previous example would work, it's going to be far less efficient, since GROMACS would need to load in the trajectory each time. 

I'll skip over most of the details, as this was discussed in the previous notebook. 

In [2]:
# path to the trajectory file
traj_path = '../../4_prod_f305/trimmed_470.xtc'

# we need to tell us what tpr file to use
tpr_path = '../../4_prod_f305/run.tpr'

# we also need to know the location of the index file
ndx_path = '../../index.ndx'


In [3]:
groups =!cat ../../index.ndx | grep '\['

#strip out the square brackets and space
for i,group in enumerate(groups):
    group = group.strip('[]')
    group = group.strip()
    groups[i] = group
    
stripped_group = []
for i in range(0, len(groups)):
    if 'Other' not in groups[i]:
        if 'System' not in groups[i]:
            if 'lipids' not in groups[i]:
                temp_group = {'name': groups[i], 'index':i}
                stripped_group.append(temp_group)
                


In [4]:
traj = md.load('../../4_prod_f305/trimmed_470.xtc', 
            top='../../4_prod_f305/confout.gro')

We will just grab the last two times to set the range for our analysis

In [50]:
filenames_ndx = []
filenames_xpm = []


for i in range(0, len(stripped_group)):
    for j in range(i, len(stripped_group)):
        index_i = stripped_group[i]['index']
        index_j = stripped_group[j]['index']
        name_i = stripped_group[i]['name']
        name_j = stripped_group[j]['name']
        print(index_i, index_j, '##', name_i, name_j)

        #not interested in water-water hbonds so we can skip them for efficiency
        if 'tip3p' not in name_i and 'tip3p' not in name_j:
            text_file = open(f'selection.txt', 'w')
            text_file.write(f'{index_i}\n{index_j}\n')
            text_file.close()

            output_file = f'hbond_{name_i}_{name_j}.xvg'
            output_ndx_file = f'hbond_{name_i}_{name_j}.ndx'
            output_xpm_file = f'hbond_{name_i}_{name_j}.xpm'


            filenames_ndx.append(output_ndx_file)
            filenames_xpm.append(output_xpm_file)


            msg = f'/usr/local/gromacs/bin/gmx hbond -f {traj_path} -s {tpr_path} -n {ndx_path} -num {output_file} -hbn {output_ndx_file} -hbm {output_xpm_file}  < selection.txt'

            print(msg)
            !{msg}

2 2 ## ucer2 ucer2
/usr/local/gromacs/bin/gmx hbond -f ../../4_prod_f305/trimmed_470.xtc -s ../../4_prod_f305/run.tpr -n ../../index.ndx -num hbond_ucer2_ucer2.xvg -hbn hbond_ucer2_ucer2.ndx -hbm hbond_ucer2_ucer2.xpm  < selection.txt
                      :-) GROMACS - gmx hbond, 2022.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /Users/cri/Dropbox/Mac (3)/Documents/Projects/CER_reverse_mapped_v3_allext/analysis_scripts/hydrogen_bonding
Command line:
  gmx hbond -f ../../4_prod_f305/trimmed_470.xtc -s ../../4_prod_f305/run.tpr -n ../../index.ndx -num hbond_ucer2_ucer2.xvg -hbn hbond_ucer2_ucer2.ndx -hbm hbond_ucer2_ucer2.xpm

Reading file ../../4_prod_f305/run.tpr, VERSION 2020.6 (single precision)
Note: file tpx version 119, software tpx version 127
Specify 2 groups to analyze:
Group     0 (         System) has 271956 elements
Group     1 (          Other) has 271956 elements
Group     2 (          ucer2) has 103200 elements
Group     

## Parsing the data

Let us use the Lipid class to make it easier to deal with the index files that are generated.

In [51]:
#a data container class to make it easier to query the system than mdtraj's datastructure
# in particular, this will allow me to calculate com of each lipid, including separately consider
# each tail in the CERs and identify hairpin vs. extended

class Lipid:
    def __init__(self, name, res_id):
        self.name = name
        self.atom_xyz = []
        self.atom_index = []
        self.atom_element = []
        self.atom_name = []
        self.res_id = res_id
        self.com = []
        self.is_hairpin = True

        self.com1 = []  #if we are a two chain lipid, we will also calculate com of each chain
        self.com2 = []
        self.headgroup_com = []
        self.carbons_chain1 = [] #if we are a two chain lipid, we need to know which indices to consider
        self.carbons_chain2 =[]
        self.headgroup = []
        self.cer_graph = nx.Graph() 
        self.angle = 0
        
        self.calc_com_performed = False
        self.calc_tails_performed = False
        self.check_hairpin_performed = False
        self.headgroup_oxygens = []

        self.layer = -1
        #for CERS
        self.layer1 = -1
        self.layer2 = -1
        
        self.orientation = 0
        
    def calc_orientation(self):
        if self.calc_com_performed == False:
            self.calc_com()

            
        if 'cer' in self.name:
            if self.calc_tails_performed == False:
                self.calc_tails()
            if self.check_hairpin_performed == False:
                self.check_hairpin()
                
            if self.is_hairpin == True:
                temp = self.com[2] -self.headgroup_com[2]
                
                if temp > 0:
                    self.orientation = 1
                else:
                    self.orientation = -1
            else:
                temp = self.com2[2] - self.com1[2]
                if temp > 0:
                    self.orientation = 1
                else:
                    self.orientation = -1
        else:
            temp_z = 0 
            for atom_id in self.headgroup_oxygens:
                temp_z = temp_z + self.atom_xyz[atom_id][2]
            temp_z = temp_z/len(self.headgroup_oxygens)

            temp = self.com[2] - temp_z
            if temp > 0:
                self.orientation = 1
            else:
                self.orientation = -1
        
    #set up arays that store the pattern for each chain
    def add_atom(self, xyz, element, index):
        self.atom_xyz.append(np.array(xyz))
        # often the element name has a digit appended to it
        # while useful for identifing atoms within the molecule, we don't need it and 
        # actually need to remove to to identify the mass
        element_stripped = "".join(filter(lambda x: not x.isdigit(), element))
        self.atom_element.append(element_stripped)
        self.atom_name.append(element)
        self.atom_index.append(index)
        if element_stripped == "O":
            self.headgroup_oxygens.append(len(self.atom_element)-1)
            
    
    def calc_tails(self):
           
        #loop over all C-C and C-H particle pairs in the residue
        for i in range(0, len(self.atom_xyz)):
            if self.atom_element[i] == 'C':
                for j in range(i+1, len(self.atom_xyz)):
                    if self.atom_element[j] == 'C' or self.atom_element[j] == 'H':
                        #note we are assuming we have an unwrapped trajectory
                        dist = np.linalg.norm(self.atom_xyz[j]-self.atom_xyz[i])
                        if dist < 0.18:
                            self.cer_graph.add_edge(i,j) #note we will use relative indices in the residue
                            
        chain_ids = []

        for c in nx.connected_components(self.cer_graph):                
            chain_temp = []
            for node in self.cer_graph.subgraph(c).nodes:
                # first let us consider the C-backbone atoms
                if 'C' in self.atom_element[node]:
                    connections = self.cer_graph.edges(node)
                    #only consider those with 4 connections
                    if len(connections) == 4:
                        C_count = 0
                        H_count = 0
                        for connection in connections:
                            temp_id = connection[1] #who we are connected to is the second element
                            if 'H' in self.atom_element[temp_id]:
                                H_count = H_count + 1
                            elif 'C' in self.atom_element[temp_id]:
                                C_count = C_count + 1
                        #we specified we need 4 connections and that they need to be either [C][C][H][H] or [C][H][H][H]
                        if C_count >= 1 and H_count == 2:
                            chain_temp.append(node) #add the base carbon to the list of atoms

                            for connection in connections:
                                temp_id = connection[1]
                                #note only add the hydrogen atoms since we looping over all Carbons already
                                if 'H' in self.atom_element[temp_id]:
                                    chain_temp.append(connection[1])
                                    
            chain_ids.append(chain_temp) 
                
        if len(chain_ids) == 2:
            self.carbons_chain1 = chain_ids[0]
            self.carbons_chain2 = chain_ids[1]
        else:
            print(self.name, " Expected two tails for cer, but we identified: ", len(chain_ids))
        for i in range(0, len(self.atom_xyz)):
            if i not in self.carbons_chain1:
                if i not in self.carbons_chain2:
                    self.headgroup.append(i)

        self.calc_tails_performed = True

                          
    def calc_com(self):
        temp_pos = [0,0,0] 
        norm = 0
        for i in range(0, len(self.atom_element)):
            temp_mass = element_from_symbol(self.atom_element[i]).mass
            norm = norm + temp_mass
            for j in range(0,3):
                temp_pos[j] = temp_pos[j] + temp_mass * self.atom_xyz[i][j]
        
        for j in range(0,3):
            temp_pos[j] = temp_pos[j]/(norm)
        self.com = temp_pos
        if self.name == 'ucer2':
            #make sure we have identified the tails 
            # call the function if we haven't yet
            if self.calc_tails_performed == False:
                self.calc_tails()
            temp_pos1 = [0,0,0] 
            norm = 0
            for i in range(0, len(self.carbons_chain1)):
                ii = self.carbons_chain1[i]
                temp_mass = element_from_symbol(self.atom_element[ii]).mass
                norm = norm + temp_mass
                for j in range(0,3):
                    temp_pos1[j] = temp_pos1[j] + temp_mass * self.atom_xyz[ii][j]
        
            for j in range(0,3):
                temp_pos1[j] = temp_pos1[j]/(norm)
            self.com1 = temp_pos1
            
            temp_pos2 = [0,0,0] 
            norm = 0
            for i in range(0, len(self.carbons_chain2)):
                ii = self.carbons_chain2[i]
                temp_mass = element_from_symbol(self.atom_element[ii]).mass
                norm = norm + temp_mass
                for j in range(0,3):
                    temp_pos2[j] = temp_pos2[j] + temp_mass * self.atom_xyz[ii][j]
        
            for j in range(0,3):
                temp_pos2[j] = temp_pos2[j]/(norm)
            self.com2 = temp_pos2
            
            temp_pos3 = [0,0,0] 
            norm = 0
            for i in range(0, len(self.headgroup)):
                ii = self.headgroup[i]
                temp_mass = element_from_symbol(self.atom_element[ii]).mass
                norm = norm + temp_mass
                for j in range(0,3):
                    temp_pos3[j] = temp_pos3[j] + temp_mass * self.atom_xyz[ii][j]
        
            for j in range(0,3):
                temp_pos3[j] = temp_pos3[j]/(norm)
            self.headgroup_com = temp_pos3
        self.calc_com_performed = True
    
    # mdtraj/mbuild/vmd all like nm as their unit; vmd does angstroms, so often useful to convert
    def scale_com(self, factor=10.0):    
        for j in range(0,3):
            self.com[j] = self.com[j]*factor
            if self.name == 'ucer2':
                self.com1[j] = self.com1[j]*factor
                self.com2[j] = self.com2[j]*factor

    def check_hairpin(self): 
        if self.calc_com_performed == False:
            self.calc_com()

        temp1 = np.array(self.com1)- np.array(self.headgroup_com)
        temp2 = np.array(self.com2)- np.array(self.headgroup_com)
        
        uv_1 = temp1/np.linalg.norm(temp1)
        uv_2 = temp2/np.linalg.norm(temp2)

        dp = np.dot(uv_1, uv_2)
        self.angle = np.arccos(dp)
        
        #hairpin if greater than 90
        if self.angle > 3.14/2:
            self.is_hairpin = False
        self.check_hairpin_performed = True
            
    def calc_layer(self, layers):
        for i, layer_t in enumerate(layers):
            if 'cer' in self.name:
                if self.com1[2] > layer_t[0] and self.com1[2] < layer_t[1]:
                    self.layer1 = i
                if self.com2[2] > layer_t[0] and self.com2[2] < layer_t[1]:
                    self.layer2 = i
            else:
                if self.com[2] > layer_t[0] and self.com[2] < layer_t[1]:
                    self.layer = i




In [52]:
lipids = []
total_lipids = 0

chain_id = 0
cer_id= 0
res_id = 0
for residue in traj[0].topology.residues:
    temp_lipid = Lipid(residue.name,res_id)
    for atom in residue.atoms:
        temp_lipid.add_atom(traj.xyz[0,atom.index,:],atom.name, atom.index)
    if 'cer' in residue.name:
        temp_lipid.calc_tails()
        temp_lipid.check_hairpin()
    temp_lipid.calc_com()
    temp_lipid.calc_orientation()

    lipids.append(temp_lipid)
    res_id = res_id+1

We will define a simple dictionary that allows us to quickly associate an atom index with the atom_name rather than dealing with the mdtraj trajectory (which is going to be slower to index into).

In [53]:
#not going to be using this at the moment
resid_from_index = {}

for lipid in lipids:
    for index in lipid.atom_index:
        resid_from_index[index] = lipid.res_id


name_from_index = {}
for lipid in lipids:
    for i,atom in enumerate(lipid.atom_index):
        name_from_index[atom] = lipid.atom_name[i] 

Let us create a few simple functions to allow us to parse the ndx and xpm files and store this information into a data container class. 

In [54]:
class Hbond:

    def __init__(self):
        self.hbonds_index = []
        self.hbonds_name = []
        self.timeseries = []
        
    
    def set_hbond(self, index_temp):
        self.hbonds_index = index_temp
        for index in index_temp:
            self.hbonds_name.append(name_from_index[index])
        
    def set_timeseries(self, temp_mat):
        for i in range(0,len(temp_mat)):
            if 'o' in temp_mat[i]:
                self.timeseries.append(True)
            else:
                self.timeseries.append(False)
    def print_time_series(self):
        for tb in self.timeseries:
            print(tb)
                
        
        
def read_ndx_xpm(filename_ndx, filename_xpm):
    hbonds = [] 
    if os.path.exists(filename_ndx):
        hbonds_file = open(filename_ndx, 'r')
        lines = hbonds_file.readlines()
        
        read_the_line = False
        for line in lines:
            if read_the_line:
                breakdown = line.split()
                hbonds_temp = [int(breakdown[0])-1, int(breakdown[1])-1,int(breakdown[2])-1] 
                hbond_temp = Hbond()
                hbond_temp.set_hbond(hbonds_temp)
                hbonds.append(hbond_temp)
            #check to see if we have reached the hbonds section of the file
            if '[ hbonds' in line:
                read_the_line = True
            #the ndx file is structured such that hbonds occur last, 
            #so we don't need to tell the code when to stop reading
            #but just in case something is appended to this file, 
            #we will tell it to stop reading if we encounter another
            #square bracket indicating another section of the index file
            elif '[ ' in line:
                read_the_line = False
    hbonds_file.close()
    count = 0    
    if os.path.exists(filename_xpm):
        hbonds_time_file = open(filename_xpm, 'r')
        lines = hbonds_time_file.readlines()

        read_the_line = True
        #xpm files are in the opposite order of the index file
        #we will read starting from the end and stop when we 
        #encounter the definitions of the y-axis

        for ll in range(len(lines)-1, 0, -1):

            if 'y-axis' in lines[ll]:
                read_the_line = False   
            if read_the_line:
                if 'o' in lines[ll]:
                    line=lines[ll].replace(',', '')
                    #each entry has quotes, we will remove them by restricting the range
                    hbonds[count].set_timeseries(line[1:-2]) 
                    count = count+1
    hbonds_time_file.close()
    return hbonds

    

In [55]:
class hbond_agg:
    
    def __init__(self, total_time):
        self.hbonds_pair = []
        self.count_pair = []
        self.total_time = total_time
        self.mean = []
        self.stdev = []
        
    def check_hbond(self, check_pair, current_time):
        in_list = False
        for i, pair in enumerate(self.hbonds_pair):
            if pair[0] == check_pair[0]:
                if pair[1] == check_pair[1]:
                    self.count_pair[i][current_time] = self.count_pair[i][current_time]+1
                    in_list = True
            elif pair[1] == check_pair[0]:
                if pair[0] == check_pair[1]:
                    self.count_pair[i][current_time] = self.count_pair[i][current_time]+1
                    in_list = True
        if in_list == False:
            self.hbonds_pair.append(check_pair)
            temp = []
            for i in range(0, self.total_time):
                temp.append(1)
            self.count_pair.append(temp)
            self.mean.append(0)
            self.stdev.append(0)

            
    def average_stdev(self):
        for i in range(0, len(self.hbonds_pair)):
            self.mean = np.mean(np.array(self.count_pair[i]))
            self.stdev = np.std(np.array(self.count_pair[i]))
            print(self.hbonds_pair[i], "\t", self.mean , "\t", self.stdev)
            
            

In [56]:
for i in range(0, len(filenames_ndx)):
    
    filename_ndx = filenames_ndx[i]
    filename_xpm = filenames_xpm[i]
    
    hbonds = read_ndx_xpm(filename_ndx, filename_xpm)
 
    print(filename_ndx)
    print("pair\t mean\t stdev")
    #when we set up the class to aggregate hbonds by type, 
    #we will need to tell it how many frames we are considering
    #so that it can initialize the arrays.  We can just get this from
    #the length of the timeseries array in any of the entries the hbonds array
    agg_hbonds = hbond_agg(len(hbonds[0].timeseries))

    for hbond in hbonds:
        for i in range(0, len(hbond.timeseries)):
            if hbond.timeseries[i]:
                agg_hbonds.check_hbond([hbond.hbonds_name[0], hbond.hbonds_name[2]], i)
    #print out the time averaged mean and stdev
    agg_hbonds.average_stdev()


hbond_ucer2_ucer2.ndx
pair	 mean	 stdev
['N1', 'O80'] 	 83.54225352112677 	 6.7600975485050885
['N1', 'O84'] 	 50.15492957746479 	 5.003232440552368
['O80', 'O84'] 	 120.65845070422536 	 8.883413432875953
['O80', 'O4'] 	 150.74647887323943 	 9.590147981438644
['O80', 'O80'] 	 39.95422535211268 	 5.2401500252611655
['O84', 'O4'] 	 163.6161971830986 	 8.770381793638704
['O84', 'O84'] 	 38.88380281690141 	 4.813846479993916
['N1', 'O4'] 	 121.26760563380282 	 6.505052741401791
['N1', 'N1'] 	 1.5246478873239437 	 0.6735115691383053
hbond_ucer2_chol.ndx
pair	 mean	 stdev
['O80', 'O3'] 	 85.3274647887324 	 7.764252177934652
['O84', 'O3'] 	 62.24295774647887 	 6.075880483197638
['N1', 'O3'] 	 12.774647887323944 	 2.89461463585417
['O3', 'O4'] 	 47.75704225352113 	 5.951753178654428
hbond_ucer2_ffa24.ndx
pair	 mean	 stdev
['N1', 'O25'] 	 26.507042253521128 	 4.47252406739482
['N1', 'O27'] 	 80.87323943661971 	 6.60863621367086
['O80', 'O25'] 	 93.17605633802818 	 8.323815565850245
['O80', 'O27